In [7]:
import pandas as pd
import datetime
import os
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import numpy as np
from dateutil.relativedelta import relativedelta
import pytz
import sys
import warnings
pd.set_option('display.max_rows', None) 


In [8]:
def get_dev_info(device, address):
    
    r = requests.post(address + "/api/auth/login",
                      json={'username': 'meazonpro@meazon.com', 'password': 'meazonpro1'}).json()
    
    # acc_token is the token to be used in the next request
    acc_token = 'Bearer' + ' ' + r['token']
    
    # get devid by serial name
    r1 = requests.get(
        url=address + "/api/tenant/devices?deviceName=" + device,
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    
    label = r1['label']
    devid = r1['id']['id']
    r1 = requests.get(
        url=address + "/api/device/" + devid + "/credentials",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    devtoken = r1['credentialsId']

    
    return devid,acc_token,label


def read_data(acc_token, devid, address, start_time, end_time, descriptors):

    r2 = requests.get(
        url=address + "/api/plugins/telemetry/DEVICE/" + devid + "/values/timeseries?keys=" + descriptors + "&startTs=" + start_time + "&endTs=" + end_time + "&agg=NONE&limit=1000000",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    
    if r2:
        df = pd.DataFrame([])
        
        for desc in r2.keys():
            df1 = pd.DataFrame(r2[desc])
            df1.set_index('ts', inplace=True)
            df1.columns = [str(desc)]
            
            df1.reset_index(drop=False, inplace=True)
            df1['ts'] = pd.to_datetime(df1['ts'], unit='ms')
            df1['ts'] = df1['ts'].dt.tz_localize('utc').dt.tz_convert('Europe/Athens')
            df1 = df1.sort_values(by=['ts'])
            df1.reset_index(drop=True, inplace=True)
            df1.set_index('ts', inplace=True, drop=True)            
            df = pd.concat([df, df1], axis=1)

        if df.empty:
            df = pd.DataFrame([])
        else:
            for col in df.columns:
                df[col] = df[col].astype('float64')
    else:
        df = pd.DataFrame([])
        # print('Empty json!')
    return df

def current_unbalance_iec(df):
    """
    Calculate the current unbalance percentage as per the IEC standard.
    """
    # Calculate the mean current
    avg_current = (df['curA']+df['curB']+df['curC']) / 3
    
    # Calculate the absolute deviations from the mean
    d1 = abs(df['curA'] - avg_current)
    d2 = abs(df['curB'] - avg_current)
    d3 = abs(df['curC'] - avg_current)
    
    # Find the maximum deviation
    max_deviation = max(d1, d2, d3)
    
    
    # Calculate the current unbalance percentage
    unbalance = (max_deviation / avg_current) * 100
    
    return unbalance



In [49]:
def preprocess(df, info, device):
    """
    Remove occurrences of voltage drops
    """
    nomcur = info.loc[info['Serial']==device,'Ampere'].values[0]
    df = df[((df['vltA']>210) & (df['vltB']>210) & (df['vltC']>210))]
    return nomcur,df

def extract_features(orig_df, device, features_dict, nomcur, info, indnum):
    features = {}
    # orig_df[(orig_df.index.hour>7) & (orig_df.index.hour<19),'daynight'] = 'day'
    orig_df['daynight'] = orig_df.index.hour.map(lambda hour: 'day' if 7 <= hour < 19 else 'night')
    
    for ph in ['A','B','C']:
        # Extract features for each phase
        # First remove loads under nominal current
        df = orig_df.copy()
        # df = df.loc[df['cur'+ph]>0.01*nomcur]
        if df.empty:
            print('Empty df')
        
        df['angle'+ph] = np.abs(df['angle'+ph])
        
        # Power features
        df['norm_pwr'+ph] = (df['pwr'+ph] - df['pwr'+ph].min()) / (df['pwr'+ph].max() - df['pwr'+ph].min())
        features['peak_to_mean'+ph] = df['norm_pwr'+ph].max() / df['norm_pwr'+ph].mean()

        daymean = df.loc[df['daynight']=='day','pwr'+ph].mean()
        nightmean = df.loc[df['daynight']=='night','pwr'+ph].mean()
        features['day_night_ratio_'+ph] = daymean/(nightmean+ 1e-6)
        features['day_night_stdratio_'+ph] = df.loc[df['daynight']=='day','pwr'+ph].std()/(df.loc[df['daynight']=='night','pwr'+ph].std() + 1e-6)
        features['p25'+ph] = df['norm_pwr'+ph].quantile(0.25)
        features['p75'+ph] = df['norm_pwr'+ph].quantile(0.75)

        # V-I angle features
        features['p25_angle_'+ph] = df['angle'+ph].quantile(0.25)
        features['p50_angle_'+ph] = df['angle'+ph].quantile(0.5)
        features['p75_angle_'+ph] = df['angle'+ph].quantile(0.75)
        
        features['angle_std'+ph] = df['angle'+ph].std()
        features['cur_std'+ph] = df['cur'+ph].std()
        
    features['label'] = info.loc[info['Serial']==device,'Label'].values[0]
    features['device'] = device
    features_dict[indnum] = features
    return features_dict

In [39]:
info = pd.read_excel('HEDNO_info.xlsx', engine='openpyxl')
info = info[info['Label']!='Unknown']

info = info[info['Label']!='PV']
devices = list(info['Serial'])


In [84]:
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

address = 'https://mi6.meazon.com'
r = requests.post(address + "/api/auth/login",
                    json={'username': 'meazonpro@meazon.com', 'password': 'meazonpro1'}).json()

acc_token = 'Bearer' + ' ' + r['token']


entityId = '47545f30-5b7f-11ee-b2c9-653b42f73605' # DEDDHE ATHINAS
r1 = requests.get(url=address + "/api/entityGroup/"+entityId+"/entities?pageSize=1000&page=0",headers={'Content-Type': 'application/json', 
'Accept': '*/*', 'X-Authorization': acc_token}).json()

interval = 1 # interval in minutes
descriptors = 'pwrA,pwrB,pwrC,curA,curB,curC,angleA,angleB,angleC,vltA,vltB,vltC'
start_time = '1725829200000'# for training
end_time = '1726434000000'
# start_time = '1726434000000'# for testing
# end_time = '1727038800000'

features_dict = {}
indnum = 0
for i in range(0,len(r1['data'])):
    assetid = r1['data'][i]['id']['id']
    assetname = r1['data'][i]['name']
    if assetname[0]!='0':  
        r2 = requests.get(url=address + "/api/relations/info?fromId="+assetid+"&fromType=ASSET",headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
        for j in range(0, len(r2)):
            device = r2[j]['toName']
            if device in devices:
                
                print(device)
                [devid, acc_token, label] = get_dev_info(device, address)
                df = read_data(acc_token, devid, address, start_time, end_time, descriptors)
                if not df.empty:
                    [nomcur, df] = preprocess(df, info, device)
                    for date in np.unique(df.index.date):
                        daily_group = df[df.index.date == date]
                        features_dict = extract_features(daily_group, device, features_dict, nomcur, info, indnum)
                        indnum += 1
                
                # search for nested devices
                r3 = requests.get(url=address + "/api/relations/info?fromId="+devid+"&fromType=DEVICE",headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
                for k in range(0, len(r3)):                    
                    device = r3[k]['toName']
                    if device in devices:
                        print(device)
                        [devid, acc_token, label] = get_dev_info(device, address)
                        df = read_data(acc_token, devid, address, start_time, end_time, descriptors)
                        if not df.empty:
                            [nomcur, df] = preprocess(df, info, device)
                            for date in np.unique(df.index.date):
                                daily_group = df[df.index.date == date]
                                features_dict = extract_features(daily_group, device, features_dict, nomcur, info, indnum)
                                indnum += 1
features_df = pd.DataFrame.from_dict(features_dict, orient='index')

# Reset the index if you want to convert the primary keys to a column
features_df.reset_index(inplace=True)
# features_df.rename(columns={'index': 'device'}, inplace=True)
# features_df = features_df.drop('device',axis=1)
features_df = features_df.drop('index',axis=1)



102.408.001992
102.408.001975
102.408.001978
102.408.001979
102.408.001991
102.408.001967
102.408.001968
102.408.001969
102.408.001970
102.408.001971
102.408.001972
102.408.001988
102.408.001946
102.408.001945
102.408.001949
102.408.001943
102.408.001944
102.408.001948
102.408.001947
102.408.001989
102.408.001957
102.408.001953
102.408.001954
102.408.001955
102.408.001956
102.408.001952
102.408.001990
102.408.001959
102.408.001960
102.408.001961
102.408.001962
102.408.001964
102.408.001965
102.408.000226
102.408.000196
102.408.000180
102.408.000181
102.408.000182
102.408.000178
102.408.000171
102.402.002103
102.408.000221
102.408.000217
102.408.000209
102.408.000202
102.408.000207
102.408.000227
102.408.000199
102.408.000222
102.408.000204
102.408.000225
102.408.000205
102.408.000793
102.408.000224
102.408.000195
102.408.000228
102.408.000194
102.408.000192
102.408.000211
102.408.000216
102.408.000229
102.408.000764
102.408.000197
102.408.000183
102.408.000172
102.408.000173
102.408.00

In [81]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, make_scorer, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import GridSearchCV


In [86]:
features_df = features_df.drop('index',axis=1)
features_df.head()

,peak_to_meanA,day_night_ratio_A,day_night_stdratio_A,p25A,p75A,p25_angle_A,p50_angle_A,p75_angle_A,angle_stdA,cur_stdA,...,day_night_stdratio_C,p25C,p75C,p25_angle_C,p50_angle_C,p75_angle_C,angle_stdC,cur_stdC,label,device
0,2.326394,1.486948,1.356359,0.203981,0.623853,7.39800,9.3400,11.21100,2.904534,32.394549,...,1.259604,0.275685,0.649947,6.1385,9.0670,11.40250,3.776074,31.826552,OK,102.408.001992
1,2.809980,1.612578,1.531805,0.132492,0.550037,7.92750,9.8160,11.92800,2.898040,42.682926,...,1.978359,0.152364,0.488000,8.7710,10.8710,12.84850,3.326084,34.719401,OK,102.408.001992
2,2.483422,1.509052,1.111146,0.166827,0.587733,7.11400,9.1540,11.09425,3.002063,33.915320,...,0.911108,0.158823,0.591923,7.8910,10.1445,13.12875,3.971063,37.287739,OK,102.408.001992
3,2.719702,1.450482,1.132424,0.173006,0.529542,8.25525,10.5235,12.92050,3.034743,33.473222,...,1.653338,0.212705,0.656431,8.9185,11.4165,14.66725,3.772623,30.713326,OK,102.408.001992
4,2.843205,1.572746,1.149617,0.141906,0.540938,6.45775,8.0975,9.98725,2.874082,39.204543,...,1.461919,0.224847,0.627035,5.9080,8.9530,12.25225,4.190081,32.450451,OK,102.408.001992


In [102]:
devices_with_label = features_df.groupby('device')['label'].apply(lambda x: x.unique()[0]).reset_index()

devices_train, devices_temp = train_test_split(devices_with_label, test_size=0.30, stratify=devices_with_label['label'], random_state=42)

# Second split: Split the temp set into 10% validation, 20% test
devices_val, devices_test = train_test_split(devices_temp, test_size=2/3, stratify=devices_temp['label'], random_state=42)

# Get the device IDs for each set
train_device_ids = devices_train['device']
val_device_ids = devices_val['device']
test_device_ids = devices_test['device']

# Filter the original dataset based on the split device IDs
train_set = features_df[features_df['device'].isin(train_device_ids)]
val_set = features_df[features_df['device'].isin(val_device_ids)]
test_set = features_df[features_df['device'].isin(test_device_ids)]

# Check the sizes to verify the split
print(f"Training set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Test set size: {len(test_set)}")

print('train',train_set['label'].value_counts())
print('train',val_set['label'].value_counts())
print('train',test_set['label'].value_counts())

train OK       336
Wrong     42
Name: label, dtype: int64
train OK       49
Wrong     7
Name: label, dtype: int64
train OK       98
Wrong    14
Name: label, dtype: int64


In [103]:
X_train = train_set.drop(columns=['label','device'])  # Dropping the target column
y_train = train_set['label']  # Target column

X_test = test_set.drop(columns=['label','device'])  # Dropping the target column
y_test = test_set['label']  # Target column

# # First split: 80% for training+test and 20% for validation
# X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# # Second split: 70% of the remaining data for training and 30% for validation
# X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, train_size=0.7, stratify=y_temp, random_state=42)
# # Split the data into training and testing sets

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, pos_label='OK'),
    'recall': make_scorer(recall_score, pos_label='OK'),
    'f1': make_scorer(f1_score, pos_label='OK')
}


# Grid search
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(class_weight={'OK': 1, 'Wrong': 10}, random_state=42), param_grid, cv=skf, scoring=scoring['f1'])
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

# Create the final model using the best parameters
rf_model = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    class_weight={'OK': 1, 'Wrong': 10},
    random_state=42
)

# Train the final model on the full training set
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          OK       1.00      1.00      1.00        98
       Wrong       1.00      1.00      1.00        14

    accuracy                           1.00       112
   macro avg       1.00      1.00      1.00       112
weighted avg       1.00      1.00      1.00       112



In [104]:
X_val = val_set.drop(columns=['label','device'])  # Dropping the target column
y_val = val_set['label']  # Target column
# Predict on the validation set
y_pred = rf_model.predict(X_val)

# Print the classification report
print("Classification Report on Validation Set:")
print(classification_report(y_val, y_pred))

# Confusion matrix to inspect individual class performance
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

Classification Report on Validation Set:
              precision    recall  f1-score   support

          OK       1.00      0.86      0.92        49
       Wrong       0.50      1.00      0.67         7

    accuracy                           0.88        56
   macro avg       0.75      0.93      0.79        56
weighted avg       0.94      0.88      0.89        56

Confusion Matrix:
[[42  7]
 [ 0  7]]


In [79]:
X = features_df.drop(columns=['label'])  # Dropping the target column
y = features_df['label']  # Target column
y_pred = rf_model.predict(X)

# Print the classification report
print("Classification Report on Validation Set:")
print(classification_report(y, y_pred))

# Confusion matrix to inspect individual class performance
print("Confusion Matrix:")
print(confusion_matrix(y, y_pred))

Classification Report on Validation Set:
              precision    recall  f1-score   support

          OK       1.00      1.00      1.00       483
       Wrong       0.98      1.00      0.99        63

    accuracy                           1.00       546
   macro avg       0.99      1.00      1.00       546
weighted avg       1.00      1.00      1.00       546

Confusion Matrix:
[[482   1]
 [  0  63]]
